In [1]:
from data_processing import load_data_2
from datetime import datetime
import pandas as pd
import json

In [3]:
#loading the original dataset
df = load_data_2('../data')

In [4]:
#mapping AreaID values to their meaning
#load the dictionary
AreaID_dict = "../data/AreaID_dict.json"
with open(AreaID_dict, 'r') as file:
    AreaID_dict = json.load(file)

#mapping the df
df2 = df.copy()
df2['AreaID'] = df2['AreaID'].map(AreaID_dict)
df2['Country'] = df2['AreaID'].str.extract(r'\((.*?)\)')
# Drop the original 'AreaID' column
df2.drop('AreaID', axis=1, inplace=True)
#dropping 'UnitName' as it's useless and rearranging the columns\
df3 = df2[['StartTime','EndTime','Country','PsrType','Load','quantity']]

#Replace NaN values with 0
df3[['Load', 'PsrType', 'quantity']] = df3[['Load', 'PsrType', 'quantity']].fillna(0)

#Filtering by green_energy PsrType
df3 = df3[df3['PsrType'].isin(["B01", "B09", "B10", "B11", "B12","B13", "B15", "B16", "B18", "B19", 0])]

/var/folders/2q/wnf6ttl1343fw9wvfvznftw0_zknrv/T/ipykernel_60646/331212565.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[['Load', 'PsrType', 'quantity']] = df3[['Load', 'PsrType', 'quantity']].fillna(0)


In [5]:
#WORKING ON TIME COLUMNS 
#in order to do so we will separate Load and quantity into different dataframes (we'll rejoin them later)
dfLoad = df3[['StartTime','EndTime','Country','Load']].groupby(['StartTime','EndTime','Country']).sum().reset_index()
dfQuantity = df3[['StartTime','EndTime','Country','PsrType','quantity']].groupby(['StartTime','EndTime','Country','PsrType']).sum().reset_index()

In [6]:
#ENERGY GENERATION DATAFRAME (dfQuantity)
# Pivot the DataFrame
dfQuantity = dfQuantity.pivot(index=['StartTime', 'EndTime'], columns=['Country', 'PsrType'], values='quantity')

# Flatten the multi-level columns
dfQuantity.columns = ['{}_{}'.format(col[0], col[1]) for col in dfQuantity.columns]
dfQuantity.drop(columns=dfQuantity.filter(like='_0', axis=1).columns, inplace=True)
dfQuantity.reset_index(inplace=True)

In [7]:
#ENERGY CONSUMPTION DATAFRAME (dfLoad)
dfLoad.rename(columns={'Load':'load'}, inplace=True)
dfLoad = dfLoad.pivot(index=['StartTime', 'EndTime'], columns=['Country'], values='load')

# Flatten
dfLoad.columns = ['{}_{}'.format(col, "load") for col in dfLoad.columns]
dfLoad.reset_index(inplace=True)

In [8]:
#MERGE BOTH DATAFRAMES
df4 = pd.merge(dfLoad, dfQuantity, on=['StartTime', 'EndTime'], how='outer')
df4 = df4[['StartTime','EndTime'] + sorted([col for col in df4.columns if col not in ['StartTime','EndTime']])]

In [11]:
#CLEAN THE DATAFRAME
df4.fillna(0, inplace=True)

In [12]:
#POPULATING THE EMPTY CELLS
df5 = df4.copy()
# Convert to datetime
df5["StartTime"] = pd.to_datetime(df5["StartTime"], format="%Y-%m-%dT%H:%M%zZ")
df5["EndTime"] = pd.to_datetime(df5["EndTime"], format="%Y-%m-%dT%H:%M%zZ")
# Set the index to 'StartTime' or 'EndTime'
df5.set_index('StartTime', inplace=True)

# Resample to hourly frequency, interpolate, and sum Load values
df6 = df5.resample('H').sum().interpolate()

/var/folders/2q/wnf6ttl1343fw9wvfvznftw0_zknrv/T/ipykernel_60646/2710564736.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df6 = df5.resample('H').sum().interpolate()


In [14]:
#df5.set_index('StartTime', inplace=True)
df6 = df5.interpolate(method='linear', limit_direction='both')

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got linear

In [ ]:
#CREATING THE SURPLUS COLUMN
df6['surplus'] = df6['quantity'] - df6['load']
df6.reset_index(inplace=True)

KeyError: 'quantity'